In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
df=pd.read_csv("../input/kiva_loans.csv")



**Loan by sector**

In [ ]:

loan_by_sector=pd.DataFrame(df.groupby(["sector"])["loan_amount"].mean().reset_index())
plt.figure(111,figsize=(7,9))
plt.style.use("ggplot")
sns.barplot(x=loan_by_sector["sector"],y=loan_by_sector["loan_amount"] ,   data=loan_by_sector)
#plt.bar(loan_by_sector["sector"],loan_by_sector["loan_amount"],color=["red","green","blue","pink","orange","yellow","c","black"],alpha=.7)
plt.gca().set_xticklabels(loan_by_sector["sector"],rotation='90')
plt.gca().set_xlabel("sectors")
plt.gca().set_ylabel("amount in USD")
plt.show()

# loan by country

In [ ]:
plt.figure(figsize=(8,8))
loan_by_country=df.groupby(["country"])["loan_amount"].agg("mean")
loan_by_country=pd.DataFrame(loan_by_country.sort_values(ascending=False)[0:25])
sns.barplot(y=loan_by_country.index,x=loan_by_country["loan_amount"],data=loan_by_country)
plt.gca().set_xlabel("average loan amount")
plt.gca().set_ylabel("Country")
plt.gca().set_title("average loan amount by country (Top 25)")
plt.show()

# loan amount by activity

In [ ]:
plt.figure(figsize=(10,10))
loan_by_activity=pd.DataFrame(df.groupby(["activity"])["loan_amount"].agg("mean"))[0:30]
sns.barplot(x=loan_by_activity["loan_amount"],y=loan_by_activity.index,data=loan_by_activity)
plt.gca().set_xlabel("Mean Amount in USD")
plt.gca().set_title("Loan amount by activity")
plt.show()

# Loan distribution in India

In [ ]:
c_r=pd.DataFrame(df.groupby(["country","region"])["loan_amount"].agg("sum"))
c_r=c_r.reset_index()
c_r_india=c_r[(c_r["country"]=="India")]

# Time to fund 

In [ ]:

#plt.figure()
time=df[["country","posted_time","funded_time"]]
time=time.copy()
time["time"]=(pd.to_datetime(time.funded_time)-pd.to_datetime(time.posted_time))
time["time"]=time["time"].apply(lambda x : x.days)
#time=time.sort_values(by="time",ascending=False)

time_taken=pd.DataFrame(time.groupby(["country"])["time"].agg("mean").sort_values(ascending=False))

time_taken=time_taken[:30]

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(y=time_taken.index,x=time_taken["time"],data=time_taken)
plt.gca().set_xlabel("Average number of days")
plt.gca().set_title("Countries v/s mean no of days for loan to be funded")
plt.show()

                

# Mean Terms in month 

In [ ]:
terms=pd.DataFrame(df.groupby(["country"])["term_in_months"].agg("mean").sort_values(ascending=False))[:50]
plt.figure(figsize=(10,10))
plt.style.use("ggplot")
sns.barplot(x=terms.term_in_months,y=terms.index,data=terms)
plt.show()

## Lenders

In [ ]:

d=df[["funded_amount","lender_count"]]
plt.figure(figsize=(7,6))
plt.scatter(x=d.lender_count,y=d.funded_amount,c="blue",alpha=.6)
plt.gca().set_ylabel("Amount in USD")
plt.gca().set_xlabel("Number of lenders")
plt.gca().spines["right"].set_visible(False)
plt.gca().spines["top"].set_visible(False)
plt.show()


## Story of Indian sectors

In [ ]:
import squarify

plt.figure(figsize=(8,8))
india=df[df["country"]=="India"]
india=pd.DataFrame(india.groupby(["sector"])["funded_amount"].agg("count"))
squarify.plot(sizes=india.funded_amount,label=india.index)
plt.show()

## world regions

In [ ]:
df2=pd.read_csv("../input/kiva_mpi_region_locations.csv")

In [ ]:
merge=df.merge(df2,how="inner")
amount=pd.DataFrame(merge.groupby(["world_region"])["funded_amount"].agg("mean").sort_values(ascending=False))
plt.figure(figsize=(10,6))
sns.barplot(x=amount.funded_amount,y=amount.index,data=amount)
plt.gca().set_xlabel("Mean funded amount in USD")
plt.gcf().subplots_adjust(left=0.25,bottom=.2)
for i, v in enumerate(amount.funded_amount):
    plt.text(0.7,i,round(v),color='k',fontsize=9)
plt.gca().spines["top"].set_visible(False)
plt.gca().spines["right"].set_visible(False)
plt.show()

## Story of south asia

In [ ]:
from mpl_toolkits.basemap import Basemap
south_asia=df2[df2['world_region']=="South Asia"][["lat",'lon']]
lat=list(south_asia["lat"])
lon=list(south_asia["lon"])

plt.figure(figsize=(14, 8))
earth = Basemap(projection='lcc',
                resolution='h',
                llcrnrlon=67,
                llcrnrlat=5,
                urcrnrlon=99,
                urcrnrlat=37,
                lat_0=28,
                lon_0=77
)
earth.drawcoastlines()
earth.drawcountries()
earth.drawstates(color='#555566', linewidth=1)
earth.drawmapboundary(fill_color='#46bcec')
earth.fillcontinents(color = 'white',lake_color='#46bcec')
# convert lat and lon to map projection coordinates
longitudes, latitudes = earth(lon, lat)
plt.scatter(longitudes, latitudes, 
            c='blue',alpha=0.5, zorder=10)
plt.savefig('Loans Disbursed in India', dpi=350)

plt.show()

## Food and Agriculture Loans

In [ ]:
food=pd.DataFrame(df[df["sector"]=="Food"].groupby(["country"])["funded_amount"].agg("count").sort_values(ascending=False))[:15]
food.columns=["count"]
agri=pd.DataFrame(df[df["sector"]=="Agriculture"].groupby(["country"])["funded_amount"].agg("count").sort_values(ascending=False))[:15]
agri.columns=["count"]
f,(ax1,ax2)=plt.subplots(1,2)
sns.barplot(x=food["count"],y=food.index,data=food,ax=ax1)
sns.barplot(x=agri["count"],y=agri.index,data=agri,ax=ax2)
ax1.set_xlabel("Food loan count")
ax2.set_xlabel("Agriculture loan count")
plt.gcf().subplots_adjust(left=0.25,bottom=.2,)
ax2.yaxis.tick_right()
ax1.spines["right"].set_visible(False)
ax1.spines["top"].set_visible(False)
ax2.spines["top"].set_visible(False)
plt.suptitle("Food and Agriculture(Top 15 countries)")
plt.show()